# Abstract:
- This dataset is intended to represent the potable ground-water resource.
- This dataset therefore does not include thermal and saline water *(temperature greater than `50℃` or dissolved solids greater than `3000 ㎎/L` or specific conductance greater than `4000 µS/㎝`)*.
- In addition, this dataset includes only the most recent arsenic analysis available for each well, and only analyses performed by hydride generation or ICP/MS.

In [1]:
import pandas as pd
import datetime
from state_names import name_this

# Basic documentation of dataset elements:
- `STAID`: USGS station identifier, based on latitude and longitude *(each well in the National Water Information System has an unique USGS station identifier)*
- `STATE`: Two-letter postal code for the U.S. state in which the sample was collected.
- `FIPS`: Federal Information Processing Standard state and county codes (see http://www.itl.nist.gov/fipspubs/fip6-4.htm )
- `LAT_DMS`: Latitude of well, *in degrees, minutes, and seconds*
- `LONG_DMS`: Longitude of well, *in degrees, minutes, and seconds*
- `WELLDPTH`: Depth *(below land surface)* of finished well *(may be less than total hole depth drilled)* ***(feet)***
- `SAMPDATE`: Date water sample was collected
- `SAMPTIME`: Time water sample was collected
- `AS_RMRK`: Remark code qualifying the analytical result in AS_CONC.
    - For example, the combination of AS_RMRK="<" and AS_CONC="1" indicates that arsenic was not detectable *at a laboratory reporting limit of 1 microgram (㎍) per liter.*
- `AS_CONC`: Concentration of arsenic in sample, in micrograms per liter (㎍/L) as arsenic
- `LAT_DD`: Latitude of well, *in decimal degrees*
- `LON_DD`: Longitude of well, *in decimal degrees*

In [2]:
# Column 1 needs to be passed in as a list in "skiprows", otherwise the header is skipped too.
df = pd.read_csv('Resources/arsenic_nov2001.csv', skiprows = [1]) #, index_col='STAID'

# Many columns *frustratingly* have spaces in their names . . . This fixes that issue.
df = df.rename(columns={'LAT_DMS ': 'LAT_DMS',
                        ' LON_DMS': 'LON_DMS',
                        'WELLDPTH  ':'WELLDPTH',
                        'SAMPDATE ':'SAMPDATE'})
df = df.fillna(value=0)
df.head()

,STAID,STATE,FIPS,LAT_DMS,LON_DMS,WELLDPTH,SAMPDATE,SAMPTIME,AS_RMRK,AS_CONC,LAT_DD,LON_DD
0,182113064451900,VI,78020,182113,644519,100.0,31-Aug-92,1020,<,1.0,18.35361,64.75528
1,182109064460300,VI,78020,182109,644603,60.0,26-Aug-92,1130,<,1.0,18.35250,64.76750
2,180859065474100,PR,72069,180859,654741,0.0,07-Jul-82,940,<,1.0,18.14972,65.79472
3,180339065523300,PR,72151,180339,655233,120.0,07-Jul-82,1645,<,1.0,18.06083,65.87583
4,175836066021400,PR,72015,175836,660214,0.0,12-Aug-86,1545,<,1.0,17.97667,66.03722


In [3]:
df['SAMPDATE'] = pd.to_datetime(df['SAMPDATE'])

In [4]:
slim_df = df[['STAID', 'SAMPDATE', 'STATE', 'LAT_DD', 'LON_DD', 'WELLDPTH', 'AS_CONC']]
slim_df = slim_df.rename(columns={'LAT_DD': 'LAT', 'LON_DD': 'LON', 'WELLDPTH': 'WELL_DEPTH'})
slim_df = slim_df.sort_values('SAMPDATE')
slim_df.head()

,STAID,SAMPDATE,STATE,LAT,LON,WELL_DEPTH,AS_CONC
6642,350348089501101,1970-01-01,TN,35.06333,89.83639,87.7,1.2
4031,353535081133401,1970-01-01,NC,35.59306,81.22611,0.0,1.0
4032,375246081133401,1970-01-01,WV,37.87944,81.22611,49.5,1.0
4033,264501081141901,1970-01-01,FL,26.75028,81.23861,0.0,1.0
4034,264508081142901,1970-01-01,FL,26.75222,81.24139,0.0,2.0


In [5]:
drop_rows = []
under_5 = []
current_st = []
st_1942 = []
for i in range(len(slim_df)):
    if slim_df.LAT[i] > 50:
        drop_rows.append(i)
    elif slim_df.LAT[i] < 24.5:
        drop_rows.append(i)
    else:
        if slim_df.AS_CONC[i] < 5:
            under_5.append(0)
            current_st.append(0)
            st_1942.append(0)
        elif slim_df.AS_CONC[i] < 10:
            under_5.append(1)
            current_st.append(0)
            st_1942.append(0)
        elif slim_df.AS_CONC[i] < 50:
            under_5.append(1)
            current_st.append(1)
            st_1942.append(0)
        else:
            under_5.append(1)
            current_st.append(1)
            st_1942.append(1)
        
slimmer_df = slim_df.drop(drop_rows).reset_index(drop=True)
slimmer_df['UNDER_5'] = under_5
slimmer_df['CURRENT_STANDARD'] = current_st
slimmer_df['1942_STANDARD'] = st_1942
slimmer_df.head()

,STAID,SAMPDATE,STATE,LAT,LON,WELL_DEPTH,AS_CONC,UNDER_5,CURRENT_STANDARD,1942_STANDARD
0,350348089501101,1970-01-01,TN,35.06333,89.83639,87.7,1.2,0,0,0
1,353535081133401,1970-01-01,NC,35.59306,81.22611,0.0,1.0,0,0,0
2,375246081133401,1970-01-01,WV,37.87944,81.22611,49.5,1.0,0,0,0
3,264501081141901,1970-01-01,FL,26.75028,81.23861,0.0,1.0,0,0,0
4,264508081142901,1970-01-01,FL,26.75222,81.24139,0.0,2.0,0,0,0


In [6]:
print(f'{len(slimmer_df)} | {len(slim_df)} |{(len(slimmer_df)/len(slim_df))*100}')

19771 | 20043 |98.64291772688719


In [7]:
for i in range(len(slimmer_df)):
    abbreviation = slimmer_df.iloc[i][2]
    state_name = name_this(abbreviation)
    slimmer_df.iat[i,2] = state_name

In [8]:
values = slimmer_df['AS_CONC'].value_counts()
values

1.00      10783
2.00       2352
3.00       1323
4.00        812
5.00        533
          ...  
460.00        1
33.60         1
4.51          1
4.70          1
23.80         1
Name: AS_CONC, Length: 375, dtype: int64

In [9]:
max_lon = -103.771556
min_lon = max_lon
max_lat = 44.967243
min_lat = max_lat
for i in range(len(slimmer_df)):
    lat = slimmer_df.iloc[i][3]
    lon = slimmer_df.iloc[i][4]
    # This IF statement here corrects any longitudes incorrectly input as a positive value.
    if (lon > 0):
        # Since the Aleutian Islands of Alaskan DO cross over the 180ᵗʰ Meridian, this check is added.
        if (lat > 51) and (lat < 56):
            # The Aleutian Islands DO NOT cross over the 165ᵗʰ Meridian East, thus 165 can be used as a cap.
            if (lon < 165):
                lon *= -1
                slimmer_df.iat[i, 4] = lon
        # This includes Guam and the The Northern Mariana Islands, which form a narrow island chain.
        if (lat > 13) and (lat < 21):
            # Because the longitude is kept narrow, we avoid confusion overlapping with Hawaii.
            if (lon > 146.5) or (lon < 144.5):
                lon *= -1
                slimmer_df.iat[i, 4] = lon
        else:
            lon *= -1
            slimmer_df.iat[i, 4] = lon
        # As a result, we maintain the integrity of our coordinates.
        # All regions in the US which are East of the Prime Meridian are accounted for, 
        # and everything else appearing East has its longitude flipped and corrected.
        
    if (lat > max_lat):
        max_lat = lat
    elif (lat < min_lat):
        min_lat = lat
    if (lon > max_lon):
        max_lon = lon
    elif (lon < min_lon):
        min_lon = lon

In [10]:
print(f'Between\t {min_lat},{min_lon}')
print(f'And\t {max_lat},{max_lon}')

Between	 25.39056,-124.28416000000001
And	 49.00861,-67.80305


In [11]:
mid_lat = ((max_lat - min_lat)/2) + min_lat
mid_lon = ((max_lon - min_lon)/2) + min_lon
print(f'{mid_lat},{mid_lon}')

37.199585,-96.04360500000001


In [12]:
len(slimmer_df)

19771

In [13]:
slim_df.to_csv('Resources/work_in_progress.csv', sep=',', header=True)

In [14]:
features_and_target_df = slimmer_df[['LAT', 'LON', 'WELL_DEPTH', 'UNDER_5','CURRENT_STANDARD', '1942_STANDARD']]
# features_and_target_df = slimmer_df[['LAT', 'LON', 'WELL_DEPTH', 'AS_CONC']]
features_and_target_df.head()

,LAT,LON,WELL_DEPTH,UNDER_5,CURRENT_STANDARD,1942_STANDARD
0,35.06333,-89.83639,87.7,0,0,0
1,35.59306,-81.22611,0.0,0,0,0
2,37.87944,-81.22611,49.5,0,0,0
3,26.75028,-81.23861,0.0,0,0,0
4,26.75222,-81.24139,0.0,0,0,0


In [15]:
features_and_target_df.to_csv('Resources/features_and_target.csv', sep=',', header=True, index=False)